In [ ]:
# -----------------------------------------------------------------------------
#  LICENSE:
#    This file is distributed under the Creative Commons
#    Attribution-NonCommercial 4.0 International License (CC BY-NC 4.0).
#    https://creativecommons.org/licenses/by-nc/4.0/
#
#    You are free to:
#      • Share — copy and redistribute the material in any medium or format
#      • Adapt — remix, transform, and build upon the material
#    under the following terms:
#      • Attribution — You must give appropriate credit, provide a link to
#        the license, and indicate if changes were made.
#      • NonCommercial — You may not use the material for commercial purposes.
#
#
#  DISCLAIMER:
#    This code is provided “AS IS,” without warranties or conditions of any kind,
#    express or implied. Use it at your own risk.
# -----------------------------------------------------------------------------

In [ ]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
#Set environment
root_path = "/path-to-dataset/MM5_Dataset"

# When using Label Studio
ls_base_url='http://192.168.0.1:8080'
ls_api_key='your-API-key'
project_id_rgb = 1 
project_id_t   = 2
project_id_uv  = 3

import sys
import importlib

modules_to_clear = ['mm5_ls_utils']  # Add the modules you want to clear from cache when updating the .py file
for module in modules_to_clear:
    if module in sys.modules:
        del sys.modules[module]
        importlib.invalidate_caches()

inc_dir = root_path+'/scripts'
if inc_dir not in sys.path:
    sys.path.append(inc_dir)

from mm5_ls_utils import *


## Read camera calibration data

In [ ]:
import cv2
import numpy as np

from mm5_ls_utils import read_calibration_params

# Usage
filename = root_path+'/MM5_CALIBRATION/def_stereocalib_THERM.yml'
ts_calibration_data = read_calibration_params(filename,'stereo')
# print(ts_calibration_data)

filename = root_path+'/MM5_CALIBRATION/def_uvcam_ori.yml'
uc_calibration_data = read_calibration_params(filename,'cam')
# print(ts_calibration_data)

## Export GT maps from label-studio projects

In [ ]:
import os
import csv
import json
import numpy as np
from PIL import Image
from label_studio_sdk.client import LabelStudio
from mm5_ls_utils import rle_to_mask  # Ensure rle_to_mask is defined in mm5_ls_utils

# -------------------------------------------------------------------------
# Configuration (Assumes these variables are defined elsewhere in your code)
# -------------------------------------------------------------------------
# ls_base_url = "YOUR_LABEL_STUDIO_BASE_URL"
# ls_api_key = "YOUR_LABEL_STUDIO_API_KEY"
# root_path = "YOUR_DESIRED_ROOT_PATH"
# project_id_rgb = YOUR_PROJECT_ID

# -------------------------------------------------------------------------
# Initialize Label Studio client
# -------------------------------------------------------------------------
client = LabelStudio(
    base_url=ls_base_url,
    api_key=ls_api_key
)

# -------------------------------------------------------------------------
# Fetch the project using the RGB project ID
# -------------------------------------------------------------------------
project = client.projects.get(project_id_rgb)

# -------------------------------------------------------------------------
# Define the destination folder for output images
# -------------------------------------------------------------------------
destination_folder = os.path.join(root_path, "ANNO_V")
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# -------------------------------------------------------------------------
# Prepare data structures
#   label_mapping: Maps label names to unique numeric IDs
#   image_masks:   Stores "object" and "class" masks per image
#   metadata:      Stores metadata about each labeled instance
# -------------------------------------------------------------------------
label_mapping = {}
image_masks = {}
metadata = {}
max_id = 0

# -------------------------------------------------------------------------
# Counters
#   annotation_count: Count of all brush label annotations
#   processed_tasks_count: How many tasks (images) actually had annotations
# -------------------------------------------------------------------------
annotation_count = 0
processed_tasks_count = 0

# -------------------------------------------------------------------------
# Fetch all tasks (with annotations) from the project
# -------------------------------------------------------------------------
tasks = client.tasks.list(project=project.id, fields='all')

# -------------------------------------------------------------------------
# Process each task and its annotations
# -------------------------------------------------------------------------
for task in tasks:
    if task.annotations:
        # If there's at least one annotation, increment processed tasks
        processed_tasks_count += 1

        # Extract the original filename and task ID
        filename = task.data.get('image', '')
        
        # Extract a shorter, processed filename
        segments = filename.split('/')
        short_filename = segments[-1]
        processed_filename = (
            f"{short_filename.split('-')[1].split('_')[0]}_"
            f"{short_filename.split('.')[-2].split('_')[-1]}"
        )
        
        # Extract the numeric image ID from the processed filename
        img_id = int(processed_filename.split('_')[0])
        
        # Keep track of how many times a label appears (for instance IDs)
        instance_id_counters = {}
        
        # ---------------------------------------------------------------------
        # Each task can have multiple annotations; process each annotation
        # ---------------------------------------------------------------------
        for annotation in task.annotations:
            for result in annotation['result']:
                if result['type'] == 'brushlabels':
                    # Count this annotation
                    annotation_count += 1

                    label_name = result['value']['brushlabels'][0]
                    label_id = result['id']
                    
                    # If it's a new label, assign a new integer ID
                    if label_name not in label_mapping:
                        max_id += 1
                        label_mapping[label_name] = max_id
                    
                    # Increment (or initialize) the counter for this label_name
                    if label_name not in instance_id_counters:
                        instance_id_counters[label_name] = 1
                    else:
                        instance_id_counters[label_name] += 1
                    
                    # Get the instance ID specific to this label
                    instance_id = instance_id_counters[label_name]
                    
                    # Decode the run-length encoding (RLE) into a binary mask
                    rle = result['value']['rle']
                    binary_mask = rle_to_mask(rle, 
                                              result['original_height'], 
                                              result['original_width'])
                    
                    # Initialize the image mask entries if needed
                    if processed_filename not in image_masks:
                        image_masks[processed_filename] = {
                            'object': np.zeros(binary_mask.shape, dtype=np.uint8),
                            'class': np.zeros(binary_mask.shape, dtype=np.uint8)
                        }
                    
                    # Set object mask = instance_id
                    image_masks[processed_filename]['object'][binary_mask == 255] = instance_id
                    # Set class mask = label_mapping[label_name]
                    image_masks[processed_filename]['class'][binary_mask == 255] = label_mapping[label_name]
                    
                    # Update metadata for this instance
                    meta_key = f"{img_id}_{label_mapping[label_name]}_{instance_id}"
                    metadata[meta_key] = {
                        'label_id': label_mapping[label_name],
                        'ls_label_id': label_id,
                        'instance_id': instance_id,
                        'img_id': img_id,
                        'label_name': label_name,
                        'file_name': processed_filename
                    }

# -------------------------------------------------------------------------
# Save each generated mask to disk
# -------------------------------------------------------------------------
for name, masks in image_masks.items():
    # (Optional) Fix specific naming if needed
    name = name.replace("8dyn", "")  # Example rename; remove if unnecessary
    
    object_image_path = os.path.join(destination_folder, f"{name}_object.bmp")
    class_image_path = os.path.join(destination_folder, f"{name}_class.bmp")
    
    # Convert arrays to images and save
    Image.fromarray(masks['object']).save(object_image_path)
    Image.fromarray(masks['class']).save(class_image_path)

# -------------------------------------------------------------------------
# Sort the label mapping by the assigned ID for consistency
# -------------------------------------------------------------------------
sorted_label_items = sorted(label_mapping.items(), key=lambda x: x[1])

# -------------------------------------------------------------------------
# Save the label mapping to CSV
# -------------------------------------------------------------------------
csv_filename = os.path.join(destination_folder, 'label_mapping.csv')
with open(csv_filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ID', 'Label Name'])
    for label, label_id in sorted_label_items:
        writer.writerow([label_id, label])

# -------------------------------------------------------------------------
# Save the label mapping to JSON
# -------------------------------------------------------------------------
json_filename = os.path.join(destination_folder, 'label_mapping.json')
with open(json_filename, 'w') as file:
    json_data = {label: label_id for label, label_id in sorted_label_items}
    json.dump(json_data, file, indent=4)

# -------------------------------------------------------------------------
# Save class names (label names) to a plain text file
# -------------------------------------------------------------------------
classes_filename = os.path.join(destination_folder, 'classes.txt')
with open(classes_filename, 'w') as file:
    for label, _label_id in sorted_label_items:
        file.write(label + '\n')

# -------------------------------------------------------------------------
# Save the metadata about each labeled instance to JSON
# -------------------------------------------------------------------------
json_meta_filename = os.path.join(destination_folder, 'label_instances.json')
with open(json_meta_filename, 'w') as file:
    json.dump(metadata, file, indent=4)

# -------------------------------------------------------------------------
print(f"Finished. Processed {processed_tasks_count} frames (tasks) "
      f"with {annotation_count} brush label annotations.")

# print(label_mapping)
# print (sorted_items)
# json_data = json.dumps(metadata, indent=4)
# print(json_data)


## Alignment example

In [ ]:
import cv2
import os
import glob
import numpy as np
from IPython.display import display
       
from mm5_ls_utils import (
    auto_tone,
    rectify_image,
    align_images_cv,
    apply_perspective_scaling_and_skew,
    align_image,
    img_merge,
    display_image_from_array,
    read_calibration_params
)
        
# ----------------------------
# Configuration and File Paths
# ----------------------------
img_folder = root_path+"/MM5_RAW/"
imgID = 69  # Example image ID

# Construct file paths using glob patterns
rgb_files    = glob.glob(os.path.join(img_folder, "RGB_0", f"{imgID}_3_*_rgb.png"))
lwir_files   = glob.glob(os.path.join(img_folder, "LWIR", f"{imgID}_*_lwir.png"))
lwir16_files = glob.glob(os.path.join(img_folder, "LWIR", f"{imgID}_*_lwir16.png"))

if not rgb_files or not lwir_files or not lwir16_files:
    print("Required images not found.")
    exit(1)

print(rgb_files)
# ----------------------------
# Load Images
# ----------------------------
rgb_img    = cv2.imread(rgb_files[0], cv2.IMREAD_COLOR)
lwir_img   = cv2.imread(lwir_files[0], cv2.IMREAD_UNCHANGED)
lwir16_img = cv2.imread(lwir16_files[0], cv2.IMREAD_UNCHANGED)

# ----------------------------
# Process Thermal Images
# ----------------------------
# Adjust tone and convert 16-bit thermal to 8-bit for processing
imgT_norm = auto_tone(lwir16_img, 0.5, 99.5, None, 2)

# Use the RGB image dimensions for further processing
imageSize = (rgb_img.shape[1], rgb_img.shape[0])

# ----------------------------
# Read Calibration Data
# ----------------------------
calib_folder = root_path+"/MM5_CALIBRATION/"
# Load calibration parameters for the thermal modality
cal_dataT = read_calibration_params(calib_folder+"def_stereocalib_THERM.yml", 'stereo')
cam_dataT = read_calibration_params(calib_folder+"def_thermalcam_ori.yml", 'cam')

# ----------------------------
# Rectify the Thermal Image
# ----------------------------
# Rectify both the LWIR image and the tone-adjusted thermal image
imgT_rectified, _   = rectify_image(lwir_img, cam_dataT['CM'], cam_dataT['D'], imageSize)
imgT8_rectified, _  = rectify_image(imgT_norm, cam_dataT['CM'], cam_dataT['D'], imageSize)

# ----------------------------
# Align Thermal to RGB
# ----------------------------
# Define alignment parameters (set to identity/zero for this minimal example)
rotT = 0.2            # rotation
udistAlpha = 0.55    # scaling
shift_x = -28        # horizontal shift
shift_y = -1         # vertical shift

# Align the rectified thermal image to the RGB frame using calibration matrices
imgT_aligned, map_x, map_y = align_images_cv(
    cal_dataT['CM1'], cal_dataT['D1'],
    cal_dataT['CM2'], cal_dataT['D2'],
    cal_dataT['R'], rotT, cal_dataT['T'],
    rgb_img, imgT_rectified, udistAlpha
)

# Optionally apply a perspective transform 
pTransT = [1.0, 1.0, 0, 0, 0, 0]
# post transformation for adjustments
imgT_transf  = apply_perspective_scaling_and_skew(imgT_aligned, pTransT[0],pTransT[1],pTransT[2],pTransT[3],pTransT[4],pTransT[5]);

# A final alignment step to adjust thermal image positioning
imgFinalT = align_image(rgb_img, imgT_transf, shift_x, shift_y, 1.0, 0.0, True)

# ----------------------------
# Merge Thermal with RGB for Visualization
# ----------------------------
# Overlay the thermal image on the RGB image using a weighted blend.
weight = 60  # RGB image weight for overlay
merged_imgT = img_merge(rgb_img, imgFinalT, weight, shift_x, shift_y, 1.0, 0.0, False, False, True)
                       
# ----------------------------
# Display Results
# ----------------------------
display_image_from_array(rgb_img, title="RGB Image")
display_image_from_array(imgFinalT, title="Aligned Thermal Image")
display_image_from_array(merged_imgT, title="Merged RGB & Thermal Image")


## Label Re-Projection (MAR)
Please note that the minimum working example below uses hard-coded correction values for depth correction. The value of reTransT must be adjusted for different positions during bulk processing, as the corresponding correction matrix is not currently provided.

In [ ]:
import cv2
import json
import os
import glob
import numpy as np
import importlib

# Clear module cache for mm5_ls_utils (useful during development)
modules_to_clear = ['mm5_ls_utils']
for module in modules_to_clear:
    if module in sys.modules:
        del sys.modules[module]
        importlib.invalidate_caches()

from mm5_ls_utils import (
    get_depth,
    process_images,
    convert_bin2gray,
    get_region_growing,
    refine_map_with_region_growing,
    apply_color_to_binary_map,
    img_merge,
    display_image_from_array,
    read_calibration_params,
    align_images_cv,
    check_color_borders,
    rectify_image
)

# -------------------------------------------------------------------------
# Configuration and Paths
# -------------------------------------------------------------------------
img_folder   = os.path.join(root_path, "MM5_RAW")
anno_folder  = os.path.join(img_folder, "ANNO_V")
calib_folder = os.path.join(root_path, "MM5_CALIBRATION")

imgID = 353  # Example image ID
do_debug = True

# Calibration file paths
cal_stereo_thermal = os.path.join(calib_folder, "def_stereocalib_THERM.yml")
cal_thermal_cam    = os.path.join(calib_folder, "def_thermalcam_ori.yml")
cal_rgb_cam        = os.path.join(calib_folder, "def_rgbcam_ori.yml")  # Optional

# -------------------------------------------------------------------------
# Load Depth Image & Annotation Metadata; Get Depth Values & Annotation Maps
# -------------------------------------------------------------------------
depth_pattern = os.path.join(img_folder, "DEPTH_0", f"{imgID}_*_depth_tr.png")
depth_files = glob.glob(depth_pattern)
if not depth_files:
    raise FileNotFoundError(f"No depth image found for pattern: {depth_pattern}")
depth_img = cv2.imread(depth_files[0], cv2.IMREAD_UNCHANGED)

json_file_path = os.path.join(anno_folder, "label_instances.json")
with open(json_file_path, "r") as f:
    metadata = json.load(f)

# get_depth returns (depthValues, imgAC_colored, imgAO_colored)
depthValues, imgAC_colored, imgAO_colored = get_depth(imgID, depth_img, anno_folder, metadata)

# -------------------------------------------------------------------------
# Load RAW RGB and Thermal Images
# -------------------------------------------------------------------------
rgb_pattern   = os.path.join(img_folder, "RGB_0", f"{imgID}_*_rgb.png")
lwir_pattern  = os.path.join(img_folder, "LWIR", f"{imgID}_*_lwir.png")

rgb_files  = glob.glob(rgb_pattern)
lwir_files = glob.glob(lwir_pattern)
if not rgb_files:
    raise FileNotFoundError(f"No RGB image found for pattern: {rgb_pattern}")
if not lwir_files:
    raise FileNotFoundError(f"No thermal image found for pattern: {lwir_pattern}")

raw_rgb_img  = cv2.imread(rgb_files[0], cv2.IMREAD_COLOR)
raw_lwir_img = cv2.imread(lwir_files[0], cv2.IMREAD_UNCHANGED)
(h_rgb, w_rgb) = raw_rgb_img.shape[:2]

# -------------------------------------------------------------------------
# Rectify Thermal Image (and use raw RGB directly)
# -------------------------------------------------------------------------
cam_dataT = read_calibration_params(cal_thermal_cam, 'cam')
rect_lwir_img, _ = rectify_image(raw_lwir_img, cam_dataT['CM'], cam_dataT['D'], (w_rgb, h_rgb))
rect_rgb_img = raw_rgb_img  # Use raw RGB if rectification is not required

# -------------------------------------------------------------------------
# Align Thermal to RGB using Stereo Calibration
# -------------------------------------------------------------------------
cal_dataT = read_calibration_params(cal_stereo_thermal, 'stereo')
rotT = 0.2          # Example rotation angle
udistAlpha = 0.55   # Example scaling factor
reTransT = [0.1, 47, -2, 1.08, 0.2, False, True, False]  # Transformation parameters
pTransT = [1.0, 1.0, 0, 0, 0, 0]  # Post perspective transform parameters

imgT_aligned, map_x, map_y = align_images_cv(
    cal_dataT['CM1'], cal_dataT['D1'],
    cal_dataT['CM2'], cal_dataT['D2'],
    cal_dataT['R'], rotT, cal_dataT['T'],
    rect_rgb_img, rect_lwir_img, udistAlpha
)
imgT_transf = apply_perspective_scaling_and_skew(imgT_aligned, *pTransT)

# -------------------------------------------------------------------------
# Check Annotation Overlap and Prepare Data for Reprojection
# -------------------------------------------------------------------------
overlapAO = check_color_borders(imgAO_colored, 30)
if do_debug:
    print("OVERLAP" if overlapAO else "NO OVERLAP")

# Initialize temporary arrays and set parameters for annotation warping
imgAC_unrecT = np.zeros((480, 640, 3), dtype=np.uint8)
imgAC_unrecUV = np.zeros((560, 720, 3), dtype=np.uint8)
valx, valy = (0.5, 0.5)
imgM = auto_tone(raw_lwir_img, 0.5, 99.5, None, 2)
reTrans = reTransT
weight = 60

if do_debug:
    display_image_from_array(imgAO_colored, "imgAO_colored")
    merged_imgT = img_merge(raw_rgb_img, imgT_transf, weight, -62, -2, 1.0, 0.0, False, False, True)
    merged_imgT = img_merge(merged_imgT, imgAO_colored, weight, 0, 0, 1.0, 0.0, False, False, True)
    display_image_from_array(merged_imgT, "merged_imgT")

# Warp the annotation maps into the thermal coordinate system
imgAC_unrecT, imgAO_unrecT = process_images(
    imgAC_colored, imgAO_colored, map_x, map_y, cam_dataT, imgM, valx, valy
)
mergedAC_imgT = img_merge(imgM, imgAC_unrecT, *reTrans)
imgAC_unrecT = apply_perspective_scaling_and_skew(imgAC_unrecT, *pTransT)

# -------------------------------------------------------------------------
# Process Each Annotated Instance for Refinement
# -------------------------------------------------------------------------
combined_instance_mapT = np.zeros_like(mergedAC_imgT, dtype=np.uint8)
combined_class_mapT = np.zeros_like(mergedAC_imgT, dtype=np.uint8)
combined_colored_mapT = np.zeros((mergedAC_imgT.shape[0], mergedAC_imgT.shape[1], 3), dtype=np.uint8)
labelMask = np.zeros((mergedAC_imgT.shape[0], mergedAC_imgT.shape[1]), dtype=np.uint8)

unknown_areas = {}
eroded_areas = {}
object_ed = {}
object_instanceMap = {}
MAX_ED = 8

for idx, (key, value) in enumerate(depthValues):
    current_id = f"{key[1]}_{key[2]}"
    instanceMap = value['binary_map']
    image_id = key[0]
    # Warp the instance map using the same mapping
    _, instanceMap = process_images(instanceMap, instanceMap, map_x, map_y, cam_dataT, imgM, valx, valy)
    instanceMapSize = np.sum(instanceMap)
    # Merge with a binary conversion for consistency
    instanceMap = img_merge(imgM, convert_bin2gray(instanceMap), 0,*reTrans[1:])
    object_instanceMap[current_id] = instanceMap

    # Compute erosion iterations based on instance map size
    ed = int(max(min(np.sqrt(instanceMapSize) / 600, MAX_ED), 1))
    object_ed[current_id] = ed
    if do_debug:
        print(f"IM size: {instanceMapSize}; ed: {ed}")

    # Compute 'unknown' regions using region growing
    unknown, eroded = get_region_growing(imgM, instanceMap, walker_iterations=1+ed, seedErode=3+ed)
    unknown_areas[current_id] = unknown
    eroded_areas[current_id] = eroded

# For each instance, refine the segmentation using information from other instances
for idx, (key, value) in enumerate(depthValues):
    current_id = f"{key[1]}_{key[2]}"
    shape = unknown_areas[current_id].shape
    other_unknown = np.zeros(shape, dtype=np.uint8)
    other_eroded = np.zeros(shape, dtype=np.uint8)

    # Merge unknown and eroded regions from other instances
    for other_id, unk_area in unknown_areas.items():
        if other_id != current_id:
            other_unknown = cv2.bitwise_or(other_unknown, unk_area)
    for other_id, erd_area in eroded_areas.items():
        if other_id != current_id:
            other_eroded = cv2.bitwise_or(other_eroded, erd_area)

    rgb_color = value['instance_colour']
    instanceMap = object_instanceMap[current_id]
    ed = object_ed[current_id]

    # Refine the instance map using region growing refinement
    debugImg, refined_map, label, gray_image, edges = refine_map_with_region_growing(
        imgM, instanceMap, other_unknown, other_eroded,
        beta=50, mode='bf', canny_threshold1=130, canny_threshold2=190,
        walker_iterations=1+ed, refined_iterations=2+ed, seedErode=1+ed,
        growDilate=3+ed, grayPctMin=0, grayPctMax=97, disableCanny=overlapAO, otherErode=3
    )

    # Update label mask and store refined map in metadata
    labelMask[label == 2] = 255
    value['label_maskT'] = refined_map.copy()

    # Apply color mapping to the refined segmentation map
    refined_map_colored = apply_color_to_binary_map(refined_map, rgb_color)

    if do_debug:
        display_image_from_array(debugImg, f"debugImg T = IM size: {instanceMapSize}; ed: {ed}")

    # Update combined maps for visualization
    combined_instance_mapT[label == 2] = key[2]
    combined_class_mapT[label == 2] = key[1]
    combined_colored_mapT[label == 2] = rgb_color

# -------------------------------------------------------------------------
# Display Combined Results (if debugging)
# -------------------------------------------------------------------------
if do_debug:
    gray_img = cv2.cvtColor(imgM, cv2.COLOR_BGR2GRAY)
    mergedAOI_imgT = img_merge(gray_img, combined_colored_mapT, reTransT[0], 0, 0, 1.0, 0.0, False, True, False)
    mergedAO_imgT = img_merge(gray_img, imgAO_unrecT, *reTransT)
    display_image_from_array(mergedAO_imgT, "mergedT")
    display_image_from_array(mergedAOI_imgT, "merged IMPROVED")
    display_image_from_array(combined_colored_mapT, "Combined Colored Map T")


## Thermal processing 16 to 24bit (DTMRE)
### With stabilised temperature
We offset measurement errors of the camera by comparing the temperature in a specific area.

In [ ]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import sys

# Clear previously loaded modules if necessary
modules_to_clear = ['mm5_t24encoding', 'mm5_ls_utils', 'mm5_d16to8.py']
for module in modules_to_clear:
    sys.modules.pop(module, None)

# Import the custom functions and classes
from mm5_d16to8 import *
from mm5_t24encoding import *

# Create the colour map 
cm = ColourMap()

# -------------------------------------------------------------------------
# Define dataset paths and parameters
# -------------------------------------------------------------------------
dataset_folder = os.path.join(root_path, "MM5_RAW")
data_folder = os.path.join(dataset_folder, "LWIR")  # Folder with depth images
test_img = "548_"          # Process only images starting with this string; leave empty to process all
doStabilize = True         # Whether to stabilize the image by offsetting values
probe_size = 9             # Size of the probe square for stabilization
base_temp = 18310          # Base temperature value used for offsetting camera fluctuations

# -------------------------------------------------------------------------
# Process each LWIR16 PNG file in the data folder
# -------------------------------------------------------------------------
for file_path in glob.glob(os.path.join(data_folder, '*lwir16.png')):
    filename = os.path.basename(file_path)
    # If test_img is defined, process only matching files
    if test_img and not filename.startswith(test_img):
        continue

    print(f"Processing file: {file_path}")
    data_array = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
    if data_array is None:
        print("Failed to load image.")
        continue

    # ---------------------------------------------------------------------
    # Stabilize the image based on the bottom-center probe square
    # ---------------------------------------------------------------------
    if doStabilize:
        height, width = data_array.shape[:2]
        if height < probe_size or width < probe_size:
            raise ValueError("Image dimensions are smaller than the probe size.")

        # Determine the bottom-center probe square indices
        start_row = height - probe_size
        start_col = width // 2 - probe_size // 2

        # Extract probe square and compute its average value
        probe_square = data_array[start_row:, start_col:start_col + probe_size]
        avg_value = int(np.mean(probe_square))
        print("Average value (int) of bottom middle probe square:", avg_value)

        # Calculate difference and adjust the entire image
        difference = base_temp - avg_value
        print("Difference to apply:", difference)
        data_array = data_array.astype(np.int32) + difference
        data_array = np.clip(data_array, 0, 65535).astype(np.uint16)

    # ---------------------------------------------------------------------
    # Convert raw depth values to temperature in Celsius and apply colour map
    # ---------------------------------------------------------------------
    # The conversion formula: (raw/64) - 273.15
    temp_celsius_array = (data_array / 64) - 273.15

    # Apply the colour map for each temperature value
    rgb_colors = np.array([cm.get_temperature_color(temp) for temp in temp_celsius_array.ravel()])
    # Reshape the resulting colors to match the original image dimensions (with 3 channels)
    data_img = rgb_colors.reshape((*data_array.shape, 3))

    # ---------------------------------------------------------------------
    # Display the processed image
    # ---------------------------------------------------------------------
    plt.imshow(data_img)
    plt.title(filename)
    plt.axis('off')
    plt.show()


## Generate coloured depth focus image (ADMRE)

In [ ]:
import sys
import importlib
import os
import glob
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.signal import find_peaks
from scipy.stats import gaussian_kde

# Clear previously loaded modules for mm5_d16to8 and mm5_depthCorrect
modules_to_clear = ['mm5_d16to8']
for module in modules_to_clear:
    sys.modules.pop(module, None)

from mm5_d16to8 import *

# -------------------------------------------------------------------------
# Configuration and Parameters
# -------------------------------------------------------------------------
dataset_folder = os.path.join(root_path, "MM5_ALIGNED")
depth_folder = os.path.join(dataset_folder, "D16")

test_img = "259"         # Process only this single image (filename (id) without extension)
debug    = True          # Debug flag for additional output
debugDepthvals = True    # Additional debug output
       # Apply inpainting to fix missing depth measurements

# Processing parameters
num_channels   = 2       # 1: single channel 8-bit; 2: dual-channel RG (B=normals)
do_normals     = True    # If True, add normals to the output image
doInpaint      = True    # inpaint
inpaint_radius = 5
ol_threshold   = 0       # minimum pixel count for depth level - pixels dropped if below. 
min_width      = 50      # peak settings
max_width      = 150     # peak settings
min_focus      = 400     # minimum distance for focus
max_focus      = 900     # maximum distance for focus
res_oof_near   = 6       # resolutin
res_oof_far    = 6
res_gap        = 2
res_focus      = 1
vd_min         = 200     # Minimum valid depth value

useKDE = True # use KDE or approximation
num_peaks = 5
num_buckets = 100 # lower values like 40 for KDE
# for approximation implementation (if useKDE=False)
sigma_value=0.6
min_height = 0.00002       # Minimum height of a peak
min_prominence = 0.00005    # Minimum prominence of a peak
min_distance = 15
# -------------------------------------------------------------------------
# Locate and Load the Test Image
# -------------------------------------------------------------------------
file_pattern = os.path.join(depth_folder, f"{test_img}.png")
files = glob.glob(file_pattern)
if not files:
    raise FileNotFoundError(f"Test image not found: {file_pattern}")
file_path = files[0]
print(f"Processing file: {file_path}")

depth_array = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
if depth_array is None:
    raise ValueError(f"Failed to load image: {file_path}")

# -------------------------------------------------------------------------
# Inpaint Missing Depth Values (If Enabled)
# -------------------------------------------------------------------------
if doInpaint:
    mask = (depth_array == 0).astype(np.uint8)
    depth_array = cv2.inpaint(depth_array, mask, inpaintRadius=inpaint_radius, flags=cv2.INPAINT_NS)

# -------------------------------------------------------------------------
# Run the Focus/Depth Processing
# -------------------------------------------------------------------------
depth_img_focus = getDepth(
    depth_array, os.path.basename(file_path), debug, ol_threshold, min_width, max_width, min_focus, max_focus, res_oof_near, res_oof_far, res_gap, res_focus, num_channels,
    num_peaks,
    num_buckets,
    sigma_value,      # sensitivity
    min_height,       # Minimum height of a peak
    min_prominence,   # Minimum prominence of a peak
    min_distance,     # Minimum number of points between peaks
    useKDE
)

# -------------------------------------------------------------------------
# Additional debug output with histograms
# -------------------------------------------------------------------------

if debugDepthvals:
    showDepth(depth_array,filename, False, True, False, inpaint_radius, ol_threshold , min_width, max_width, min_focus, max_focus
                            , res_oof_near, res_oof_far, res_gap, res_focus, num_channels,num_peaks, num_buckets,
    sigma_value,      # sensitivity
    min_height,       # Minimum height of a peak
    min_prominence,   # Minimum prominence of a peak
    min_distance,     # Minimum number of points between peaks
    useKDE)
    valid_depths = depth_array[depth_array > 0]

    print(f"Depth min/max: {valid_depths.min()}/{valid_depths.max()}")
    valid_depths = depth_array[depth_array > 0]

    # Calculate min and max from non-zero values
    if valid_depths.size > 0:
        depth_min = valid_depths.min()
        depth_max = valid_depths.max()
    else:
        depth_min = None  # No valid depths available
        depth_max = None

    # Calculate how many values are within the range 1-100
    count_in_range = np.sum((depth_array >= 1) & (depth_array <= vd_min))

    # Print detailed debug information
    print(f"Depth min/max (excluding zeros): {depth_min}/{depth_max}")
    print(f"Number of values in the range 1-100: {count_in_range}")
    range_mask = (depth_array >= 1) & (depth_array <= vd_min)

    # Check if there are any values in the range
    if np.any(range_mask):
        plt.figure(figsize=(10, 6))  # Set the figure size for better visibility
        plt.imshow(range_mask, cmap='gray')  # Use a gray colormap to show the mask
        plt.colorbar()  # Add a colorbar to help identify the mask
        plt.title('Mask of Depth Values Between 1 and 100')
        plt.xlabel('X Pixel')
        plt.ylabel('Y Pixel')
        plt.show()

# -------------------------------------------------------------------------
# Merge Normals if Needed and Display
# -------------------------------------------------------------------------
title_pf = " +N" if do_normals else ""

if num_channels == 1:
    # Single-channel grayscale depth
    depth_gray = Image.fromarray(depth_img_focus, mode='L')  # 'L' for 8-bit grayscale
    if do_normals:
        nrm = normals_to_grayscale(depth_array, sobel_kernel_size=7, smoothing_kernel_size=(7, 7)) * 0.8
        nrm = np.clip(nrm, 0, 255).astype(np.uint8)
        nrm_img = Image.fromarray(nrm, mode='L')
        depth_img = Image.blend(depth_gray, nrm_img, alpha=0.3)
    else:
        depth_img = depth_gray

    # Display the single-channel image
    plt.imshow(depth_img, cmap='gray')
    plt.title('Focus Depth 8-bit' + title_pf)
    plt.colorbar(label='Depth')

elif num_channels == 2:
    # Dual-channel RG image (blue=0) plus optional normals in B channel
    depth_img_rg = np.zeros((depth_img_focus.shape[0], depth_img_focus.shape[1], 3), dtype=np.uint8)
    depth_img_rg[..., 0:2] = depth_img_focus[..., 0:2]
    if do_normals:
        nrm = normals_to_grayscale(depth_array, sobel_kernel_size=7, smoothing_kernel_size=(9, 9)) * 0.8
        nrm = np.clip(nrm, 0, 255).astype(np.uint8)
        depth_img_rg[..., 2] = nrm

    depth_img = Image.fromarray(depth_img_rg)
    plt.imshow(depth_img)
    plt.title('Focus Depth Dual Channel RG' + title_pf)

plt.show()

print("Focused depth image processed successfully!")
